# PyTorch Workflow

Let's explore an example PyTorch end-to-end worflow.

In [5]:
what_were_covering = {
    1: "data {prepare and load}",
    2: "build model",
    3: "fitting the model to data (training)",
    4: "making predictions",
    5: "saving and loading the model",
    6: "putting it all together"
}

what_were_covering

{1: 'data {prepare and load}',
 2: 'build model',
 3: 'fitting the model to data (training)',
 4: 'making predictions',
 5: 'saving and loading the model',
 6: 'putting it all together'}

In [7]:
import torch
from torch import nn ## nn contains all of PyTorch's building blocks for neural netowrks
import matplotlib.pyplot as plt

# check PyTorch version
torch.__version__

'1.13.1'

## Data (preparing and loading)

Data can be almost anything... in machine learning.

* Excel spreadsheet
* Images of nay kind
* Videos (YouTube has lots of data...)
* Audio like songs or podcasts
* DNA
* Text

Machine learning is a game of two parts:
1. Get data into a numerical representation.
2. Build a model to learn patterns in that numerical presentation.